<a href="https://colab.research.google.com/github/vvikasreddy/lexically_constrained_beam_search_/blob/main/beam_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:

Marian MT model : https://huggingface.co/docs/transformers/model_doc/marian

Code to get the logits : https://huggingface.co/docs/transformers/main_classes/output

to get the BOS and EOS tokens: https://huggingface.co/docs/transformers/main_classes/configuration#transformers.PretrainedConfig.decoder_start_token_id

get topk values : https://pytorch.org/docs/stable/generated/torch.topk.html

ideas and core implementation drawn from this paper: https://arxiv.org/pdf/1704.07138

reference to link google colab with .py file from git : https://colab.research.google.com/github/jckantor/cbe61622/blob/master/docs/A.02-Downloading_Python_source_files_from_github.ipynb


## downloading essential modules

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Importing necessary libraries

In [2]:
import torch, random
from datasets import load_dataset
from transformers import MarianMTModel, MarianTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

## loading the dataset, considering the wmt turkish - english translation

In [3]:
ds = load_dataset("wmt/wmt16", "tr-en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/36.8M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/153k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/466k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/205756 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

## Glancing the organization of the dataset

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 205756
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1001
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
})

In [5]:
ds['train'][0]

{'translation': {'en': "Kosovo's privatisation process is under scrutiny",
  'tr': "Kosova'nın özelleştirme süreci büyüteç altında"}}

## Loading the tokenizer and model, based of Marian-NMT

In [6]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tr-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-tr-en")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/840k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [7]:
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62389, 512, padding_idx=62388)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62389, 512, padding_idx=62388)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [8]:
model.config.num_beams

6

In [9]:
# print(generate_translation(ds['validation'][1]['translation']['tr']))

In [10]:
ds["validation"][1]["translation"]["tr"]

"Norveç'in beş milyon insanı en yüksek yaşam standartlarının tadını çıkarıyor, sadece Avrupa'da değil, dünyada."

In [11]:
ds["validation"][1]["translation"]["en"]

"Norway's five million people enjoy one of the highest standards of living, not just in Europe, but in the world."

## Extracting the constraints

In [13]:
# code to import constraints and store in a local directory, from my git

user = "vvikasreddy"
repo = "lexically_constrained_beam_search_"
pyfile = "constraints.py"

# i.e url is "https://github.com/vvikasreddy/lexically_constrained_beam_search_/blob/main/constraints.py"

url = f"https://raw.githubusercontent.com/{user}/{repo}/main/{pyfile}"
!wget --no-cache --backups=1 {url}

import constraints

--2024-12-06 03:13:48--  https://raw.githubusercontent.com/vvikasreddy/lexically_constrained_beam_search_/main/constraints.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4914 (4.8K) [text/plain]
Failed to rename constraints.py to constraints.py.1: (2) No such file or directory
Saving to: ‘constraints.py’

constraints.py      100%[===================>]   4.80K  --.-KB/s    in 0s      

2024-12-06 03:13:48 (49.5 MB/s) - ‘constraints.py’ saved [4914/4914]



In [14]:
# takes almost 4 minutes to get the constraints, you will see 3 progress bars
c = constraints.get_constraints()

100%|██████████| 26221852/26221852 [00:46<00:00, 560973.00it/s]


In [15]:
print("some of the constraints are :")

# Extract 5 random keys
random_keys = random.sample(list(c.keys()), 5)

for key in random_keys:
  print(key, c[key])

print("The length of the constraints is", len(c))

some of the constraints are :
('ve', 'yolsuzlukla') (('crime', 'and'), 0.9197770457991424)
('üç', 'yıl') (('three', 'years'), 0.9362123150855923)
("Nikolovski'nin", 'haberi') (('Nikolovski', 'for'), 0.9790345418288867)
('günü', "Brüksel'de") (('Brussels', 'on'), 0.9245099014389957)
('savaş', 'suçu') (('war', 'crimes'), 1.1965456966123387)
The length of the constraints is 570





    """
    Returns decoder_input_tokens, probs, vis_data

    Args:
        src_text (str): The source text to translate
        decoder_input (torch.Tensor, optional): Decoder tokens. Defaults to [].
        probabilities (torch.Tensor, optional): Corresponding decoder token probabilities. Defaults to [].
        get_constrained_token_probability (int, optional): Constraint token index. Defaults to -1.
        k (int, optional): Number of beams to generate. Defaults to 5.
        device (torch.device, optional): Device to run the model on. Defaults to None (auto-detect).

    Returns:
        tuple: decoder_input_tokens, probs, vis_data
    """
  

In [18]:
def get_ngrams(src, n = 2, ):

  src = src.split(" ")
  src = [tuple(src[i:i+n]) for i in range(len(src) - n + 1)]

  return src

def constraints_tokens(src, c):
  ngrams = get_ngrams(src)
  constraints_src = []
  for ngram in ngrams:
    # print(ngram)
    if ngram in c:
      f = c[ngram][0]
      for gram in f:

        if  gram in constraints_src: continue
        out = tokenizer(gram, return_tensors="pt")
        constraints_src.append(out["input_ids"])
        # print(constraints_src)
  return constraints_src

constraints_tokens("Southeast European Times için Priştine'den Muhamet Brayşori'nin haberi -- 21/03/12", c)

[tensor([[3113,   56,   47, 1517,    0]]),
 tensor([[5827, 1786,  373,    0]]),
 tensor([[5827, 1786,  373,    0]]),
 tensor([[3762,    0]]),
 tensor([[27,  0]]),
 tensor([[3113,   56,   47, 1517,    0]]),
 tensor([[21,  0]]),
 tensor([[ 4388, 10158,   204,     0]]),
 tensor([[1417,    0]]),
 tensor([[6041,   47, 2628,    0]]),
 tensor([[27,  0]]),
 tensor([[3113,   56,   47, 1517,    0]])]

In [ ]:
# x = constraints_tokens(ds['validation'][1]['translation']['tr'] + ' Times' +  ' için', c)
# # c
# print(x)
# print(get_input_ids(x))

In [ ]:
def visualize_data(decoder_input):
  return tokenizer.decode(decoder_input.squeeze(), skip_special_tokens = True)

In [ ]:
import torch

def get_top_k_prob(A, B, k=2):

  d = {}
  # cummulative sum
  for indx, val in enumerate(B):

    cum_sum = torch.prod(val)
    d[cum_sum] = indx

  sorted_keys = sorted(d.keys(), reverse = True)

  top_k_indices = []
  top_k_sequences = []

  for key in sorted_keys[:k]:
    top_k_indices.append(A[d[key]])
    top_k_sequences.append(B[d[key]])

  return top_k_sequences, top_k_indices

# sanity
k = 2

# A = [torch.tensor([[62388,   626,    13]]), torch.tensor([[62388,   626,     9]]), torch.tensor([[62388,   626,  1341]]), torch.tensor([[62388,   626,    27]])]
# B = [torch.tensor([[1.0000, 0.0038, 0.2500]]), torch.tensor([[1.0000, 0.0038, 0.0619]]), torch.tensor([[1.0000, 0.0038, 0.0474]]), torch.tensor([[1.0000, 0.0038, 0.0425]])]


A = [torch.tensor([[62388,  1969]]), torch.tensor([[62388,   323]]), torch.tensor([[62388,    67]]), torch.tensor([[62388,  1132]]), torch.tensor([[62388,   626]])]
B = [torch.tensor([[1.0000, 0.8746]]), torch.tensor([[1.0000, 0.0156]]), torch.tensor([[1.0000, 0.0114]]), torch.tensor([[1.0000, 0.0039]]), torch.tensor([[1.0000, 0.0038]])]

top_sequences, indices = get_top_k_prob(A, B, k)
print(f"Top {k} sequences:", top_sequences)
print("Their indices:", indices)

Top 2 sequences: [tensor([[1.0000, 0.8746]]), tensor([[1.0000, 0.0156]])]
Their indices: [tensor([[62388,  1969]]), tensor([[62388,   323]])]


In [54]:
# since wmt dataset is not domain speicific, I am trying to only take the sentences, which have constraints defined.

count = 0
indices = []
count = 1000
i = 0

while count:
  x = ds["train"][i]["translation"]["tr"]
  if constraints_tokens(x, c):
    count -=1
    indices.append(i)
  i+=1

print("The last index in all of the 3000 is,", indices[-1])

The last index in all of the 3000 is, 2102


In [55]:
len(ds["train"])
import torch
from torch.utils.data import Dataset

class Results(Dataset):

  def __init__(self, ds, c, indices):
      """
      Args:
          data : represents the turkish sentences to be passed
      """
      self.data = ds
      self.c = c
      self.indices = indices

  def generate_translation(self, src_text, decoder_input=[], probabilities=[], get_constrained_token_probability=-1, k=5, device=None):
    # print("he")
    # Auto-detect device if not specified
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move model to the specified device
    model.to(device)

      # Tokenize input
    encoder_inputs = tokenizer(src_text, return_tensors="pt").to(device)

    # If decoder_input is empty, include the decoder start token
    if len(decoder_input) == 0:

      # Initial decoder start token has probability 1
      probabilities = torch.tensor([[1.0]]).to(device)
      decoder_input = torch.tensor([[model.config.decoder_start_token_id]]).to(device)
    else:
      # Ensure decoder_input and probabilities are on the correct device
      # print(probabilities, "am in else")
      # print(decoder_input, "am in else")
      decoder_input = decoder_input.to(device)
      if probabilities != [] :
        probabilities = probabilities.to(device)

    # Change the model to eval mode and stop the computation of gradients
    model.eval()
    with torch.no_grad():
        # Generate tokens
      outputs = model(
          input_ids=encoder_inputs.input_ids,
          attention_mask=encoder_inputs.attention_mask,
          decoder_input_ids=decoder_input
      )

    # Get the most frequently generated token
      next_token_logits = outputs.logits[:, -1, :]

      # Constraint handling
      if get_constrained_token_probability != -1:
          softmax_ = torch.softmax(next_token_logits, dim=-1)
          return softmax_[0][get_constrained_token_probability]

      # Get the top k tokens with maximum logits value
      top_probs, top_indices = torch.topk(torch.softmax(next_token_logits, dim=-1), k=k)

    # Prepare output containers
    decoder_input_tokens = []
    probs = []
    vis_data = []

    for indx, id in enumerate(top_indices[0]):
        # Concatenate new tokens and probabilities
        new_decoder_input = torch.cat([decoder_input, id.unsqueeze(0).unsqueeze(0)], dim=1)
        new_probs = torch.cat([probabilities, top_probs[0][indx].unsqueeze(0).unsqueeze(0)], dim=1)

        decoder_input_tokens.append(new_decoder_input)
        probs.append(new_probs)

        # Generate visualization data
        vis_data.append((vis_data, tokenizer.decode(new_decoder_input.squeeze(), skip_special_tokens=True)))

    return decoder_input_tokens, probs, vis_data

  def get_the_text(self, x):
    token_id_list = x  # Extract the list of tensors

    # Move all tensors to CPU and convert to a list of lists
    token_ids_batch = [tensor_item.cpu().squeeze().tolist() for tensor_item in token_id_list]

    # Batch decode token IDs into sentences
    decoded_sentences = tokenizer.batch_decode(token_ids_batch, skip_special_tokens=True)

    return decoded_sentences
  def get_top_k_prob(self, A, B, k=2):
    d = {}
    # cummulative sum
    for indx, val in enumerate(B):

      cum_sum = torch.prod(val)
      d[cum_sum] = indx

    sorted_keys = sorted(d.keys(), reverse = True)

    top_k_indices = []
    top_k_sequences = []

    for key in sorted_keys[:k]:
      top_k_indices.append(A[d[key]])
      top_k_sequences.append(B[d[key]])

    return top_k_sequences, top_k_indices

  def constraints_tokens(self, src):
    ngrams = get_ngrams(src)
    constraints_src = []
    for ngram in ngrams:
      # print(ngram)
      if ngram in self.c:
        f = self.c[ngram][0]
        for gram in f:

          if  gram in constraints_src: continue
          out = tokenizer(gram, return_tensors="pt")
          constraints_src.append(out["input_ids"])
          # print(constraints_src)
    return constraints_src

  def beam_search(self, maxlen, numC, k, src, constrained_tokens):

    decoder_start_token = model.config.decoder_start_token_id

    # initialize the grids
    grids = [[[] for _ in range(numC + 1)] for _ in range(maxlen + 1)]
    probs_grid  = [[[] for _ in range(numC + 1)] for _ in range(maxlen + 1)]

    # intialize the first grid to start hyp
    grids[0][0] = [1]

    # remove during testsrc
    # constrained_tokens = get_input_ids(constraints_tokens(, constraints))
    # temporary
    # constrained_tokens = constraints_tokens("Southeast European Times için Priştine'den Muhamet Brayşori'nin haberi -- 21/03/12", constraints)

    generated_constraint_index = 0

    for t in range(1, maxlen):

        index_c = max(0, (numC - t) - maxlen)

        for c in range(index_c, min(t, numC) + 1):

            # Prepare batched generation to reduce individual calls
            s = []
            g = []

            # storing decoder inputs
            decoder_inputs = []
            probs = []
            vis_data = []

            # Batch generation of translations for current hypotheses
            for indx, element in enumerate(grids[t-1][c]):

              # guess there is no need for conditioning, just generate.
              if type(element) == int:
                decoder_input = []
                prev_probs =[]
              else:
                decoder_input = element.cuda()  # Move to GPU
                prev_probs = probs_grid[t-1][c][indx].cuda()  # Move to GPU

              # Batch collection of translations
              t_g, t_probs, t_vis_data = self.generate_translation(src_text=src, decoder_input=decoder_input, probabilities=prev_probs)

              # Extend lists more efficiently
              g.extend(t_g)
              probs.extend(t_probs)
              vis_data.extend(t_vis_data)

            # retrieve the probability of the constraint and add that to the decoder_input.
            if c > 0 and constrained_tokens:

              for indx, element in enumerate(grids[t-1][c-1]):

                if c == 1 and t == 1:
                  decoder_inputs = torch.tensor([[model.config.decoder_start_token_id]]).cuda()  # Move to GPU
                  prob = torch.tensor([[1]]).cuda()  # Move to GPU
                else:
                  decoder_inputs = element.cuda()  # Move to GPU
                  prob = probs_grid[t-1][c-1][indx].cuda()  # Move to GPU

                # print(constrained_tokens[c - 1])

                # iterating, because a constraint can be made up of many token ids
                partial_constraints = constrained_tokens[c - 1].tolist()
                # print(partial_constraints, "gandu")
                for partial_constraint in partial_constraints[0]:
                  if partial_constraint == 0: continue
                  # print(partial_constraint, "tryint to dissolve")
                  cons = self.generate_translation(src, decoder_input=decoder_input, get_constrained_token_probability=partial_constraint)
                  # print(cons, "cons", decoder_inputs, torch.tensor(partial_constraint).unsqueeze(0).unsqueeze(0))
                  # Concatenate constraints
                  decoder_inputs = torch.cat([decoder_inputs, torch.tensor(partial_constraint).unsqueeze(0).unsqueeze(0).cuda()], dim=1)
                  prob = torch.cat([prob, torch.tensor(cons).unsqueeze(0).unsqueeze(0).cuda()], dim=1)

                  g.append(decoder_inputs)
                  probs.append(prob)

            # Select top-k hypotheses
            probs_grid[t][c], grids[t][c] = self.get_top_k_prob(g, probs, k)
    # print(len(grids), grids)
    return self.get_top_k_prob(grids[maxlen -1][numC], probs_grid[maxlen - 1][numC], k = 1), self.get_top_k_prob(grids[maxlen -1][0], probs_grid[maxlen - 1][0], k = 1)

  def __len__(self):
      """Returns the total number of samples in the dataset."""
      return len(self.indices)

  def __getitem__(self, idx):
      """
      Retrieve a single sample and its label.

      Args:
          idx (int): Index of the sample to retrieve.

      Returns:
          tuple: (sample, label) where sample is the data and label is the corresponding label.
      """
      # print(self.indices[idx])
      sample = self.data["test"][self.indices[idx]]["translation"]["tr"]
      constraints = self.constraints_tokens(sample)
      # print(constraints)
      prediction, prediction_without_constraints = self.beam_search(maxlen= 30, numC=len(constraints), k = 4, src = sample, constrained_tokens = constraints)
      # print(prediction)
      prediction = self.get_the_text(prediction[1])
      prediction_without_constraints = self.get_the_text(prediction_without_constraints[1])
      result = self.data["test"][self.indices[idx]]["translation"]["en"]

      return sample, prediction[0], prediction_without_constraints[0], result

In [56]:
results = Results(ds,c, indices = indices)
results[100]
len(results)

1000

In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu

# Initialize lists to store predictions and actual texts
predictions = []
predictions_without_constraints = []
references = []

dataloader = DataLoader(results, batch_size=10, shuffle=False)

# Iterate through the entire dataloader
for tr_text, pred_text, pred_cons_text, actual_text in tqdm(dataloader):

  predictions.extend([ref.split() for ref in pred_text])
  predictions_without_constraints.extend([ref.split() for ref in pred_cons_text])
  references.extend([[ref.split()] for ref in actual_text])



  0%|          | 0/100 [00:00<?, ?it/s]<ipython-input-55-d21d833d4df2>:201: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prob = torch.cat([prob, torch.tensor(cons).unsqueeze(0).unsqueeze(0).cuda()], dim=1)
  1%|          | 1/100 [00:56<1:33:51, 56.89s/it]

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
import nltk

def calculate_bleu_score(references, translations):
    """
    Calculate BLEU score for multiple references and translations.

    Args:
    - references: List of reference translations
      (each reference is a list of lists of tokens)
    - translations: List of machine translations
      (each translation is a list of tokens)

    Returns:
    - Average BLEU score
    """
    bleu_scores = []

    for ref, trans in zip(references, translations):
        # Calculate BLEU score for this sentence
        score = sentence_bleu(ref, trans)
        bleu_scores.append(score)


    # Calculate and return average BLEU score
    return sum(bleu_scores) / len(bleu_scores)


bleu_score_constraints = calculate_bleu_score(references, predictions)
bleu_score_without_constraints = calculate_bleu_score(references, predictions_without_constraints)
print(f"BLEU Score with constraints: {bleu_score_constraints:.4f}, BLEU score without constraints: {bleu_score_without_constraints:.4f}")

In [98]:
import locale
print(locale.getpreferredencoding())
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

reference to get this working : https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working

ANSI_X3.4-1968


In [100]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=75fab2b819bf7bfff7190c78f55c39556f1e8cc71c87beb6960c94eb11a25baa
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer

def calculate_rouge_score(references, translations):
    """
    Calculate average ROUGE-1, ROUGE-2, and ROUGE-L F-scores for multiple references and translations.

    Args:
    - references: List of reference translations
      (each reference is a list of lists of tokens; we'll use the first reference set if multiple)
    - translations: List of machine translations
      (each translation is a list of tokens)

    Returns:
    - A dictionary with average ROUGE-1, ROUGE-2, and ROUGE-L F-scores.
    """
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    for ref_list, trans in zip(references, translations):
        # If you have multiple references per sentence, we take the first one
        # ref_list is something like: [["this", "is", "a", "reference"], ...]
        ref_str = " ".join(ref_list[0])  # first reference
        trans_str = " ".join(trans)

        scores = scorer.score(ref_str, trans_str)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)

    return {
        'rouge1': sum(rouge1_scores) / len(rouge1_scores),
        'rouge2': sum(rouge2_scores) / len(rouge2_scores),
        'rougeL': sum(rougeL_scores) / len(rougeL_scores)
    }

# Example usage (assuming `references` and `predictions` and `predictions_without_constraints` are defined):
rouge_scores_constraints = calculate_rouge_score(references, predictions)
rouge_scores_no_constraints = calculate_rouge_score(references, predictions_without_constraints)

print(f"ROUGE-1 with constraints: {rouge_scores_constraints['rouge1']:.4f}, ROUGE-1 without constraints: {rouge_scores_no_constraints['rouge1']:.4f}")
print(f"ROUGE-2 with constraints: {rouge_scores_constraints['rouge2']:.4f}, ROUGE-2 without constraints: {rouge_scores_no_constraints['rouge2']:.4f}")
print(f"ROUGE-L with constraints: {rouge_scores_constraints['rougeL']:.4f}, ROUGE-L without constraints: {rouge_scores_no_constraints['rougeL']:.4f}")
